In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Inspecting Case Data

## CaseViewerWidget

When working in a Jupyter notebook, you can quickly get a plot of the data you have recorded from your analysis using a `CaseViewerWidget`.


Here we show an example that uses the [Sellar MDA](../../basic_user_guide/multidisciplinary_optimization/sellar_opt.ipynb) model from the
Basic User Guide. 

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("case_viewer_widget_sellar_src_code", get_code("openmdao.test_suite.components.sellar_feature.SellarMDA"), display=False)

:::{Admonition} `SellarMDA` class definition 
:class: dropdown

{glue:}`case_viewer_widget_sellar_src_code`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarMDA

import numpy as np

# define Sellar MDA problem
prob = om.Problem(model=SellarMDA())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9, disp=False)

prob.setup()

# add recorder to the driver, model and cycle solver
recorder = om.SqliteRecorder('sellar.sql')
prob.driver.add_recorder(recorder)
prob.model.add_recorder(recorder)
prob.model.cycle.nonlinear_solver.add_recorder(recorder)

# run the optimization
prob.model.set_solver_print(level=1)
prob.run_driver()
prob.cleanup()

Having recorded our driver iterations and final problem state into the file "sellar.sql", we can now instantiate a `CaseViewerWidget` to quickly and easily generate plots of our output variables across iterations.

Using the `Source` selector we can choose to inspect either the cases recorded by the driver, the model or the cycle solver.  Having chosen the source, we can then select the range of `Cases` to inspect and then finally the `Output` variables to plot.  Hold the CTRL or Shift key when clicking choosing variables for the Y axis to select multiple outputs.

In [ ]:
om.CaseViewerWidget('sellar.sql', source='driver', y_axis=['x', 'z', 'con1', 'con2', 'obj']);

Note that if you narrow the case range to a single case, you have the ability to select an output variable for the X axis instead of `Iterations`. As an example of when this might be useful, see the following use case that implements a polynomial fit component.  We show how the curve fit might be shown using data from the model and how to use the `CaseViewerWidget` to quickly get a similar plot.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("case_viewer_widget_polyfit_src_code", get_code("openmdao.test_suite.components.polynomial_fit.PolynomialFit"), display=False)

:::{Admonition} `PolynomialFit` class definition 
:class: dropdown

{glue:}`case_viewer_widget_polyfit_src_code`
:::

In [ ]:
import numpy as np

import openmdao.api as om

from openmdao.test_suite.components.polynomial_fit import PolynomialFit


N_cp = 50
N_predict = 1000

p = om.Problem()

fit = p.model.add_subsystem('fit', PolynomialFit(N_cp=N_cp, N_predict=N_predict))
fit.nonlinear_solver = om.NewtonSolver(solve_subsystems=False, maxiter=10, iprint=2)
fit.linear_solver = om.DirectSolver()

p.setup()


np.random.seed(0)

X_cp = np.linspace(0, np.pi, N_cp)
Y_cp = np.sin(X_cp) + (np.random.random(N_cp) - 0.5)*0.1

X_true = np.linspace(0, np.pi, N_predict)
Y_true = np.sin(X_true)

p['fit.x_cp'] = X_cp
p['fit.y_cp'] = Y_cp
p['fit.x'] = X_true

p.model.add_recorder(om.SqliteRecorder("polyfit.sql"))

p.run_model()

In [ ]:
# plot curve fit using data from the model

import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.plot(p['fit.x'], p['fit.y'], label='fit.y')
ax.scatter(p['fit.x_cp'], p['fit.y_cp'], label='fit.y_cp', c='C1')

plt.legend(loc='best')
plt.show()

In [ ]:
# plot curve fit using data from the case recording

om.CaseViewerWidget("polyfit.sql", x_axis='fit.x', y_axis='fit.y');

```{Note}
    If you choose variables for the X and/or Y axes that cannot be plotted, e.g. due to dimensionality,
    a message will be displayed indicating why a plot cannot be generated.
```